# 2.2 从头训练dna gpt2大模型

### GPT-2 模型简介

GPT-2（Generative Pre-trained Transformer 2）是由 OpenAI 开发的一种基于 Transformer 架构的大型语言模型。它是一种生成式模型，意味着它可以生成连贯且上下文相关的文本。GPT-2 通过大量的无监督预训练数据学习了语言的统计规律，从而能够在多种自然语言处理任务中表现出色，如文本生成、问答、翻译等。

### 参数规模

GPT-2 提供了多个不同大小的版本，每个版本都有不同的参数量和计算资源需求。以下是主要版本及其参数规模：

1. **GPT-2 Small**：
   - 参数量：1.54 亿个参数
   - 层数：12 层
   - 隐藏层维度：768
   - 注意力头数：12

2. **GPT-2 Medium**：
   - 参数量：3.45 亿个参数
   - 层数：24 层
   - 隐藏层维度：1024
   - 注意力头数：16

3. **GPT-2 Large**：
   - 参数量：7.74 亿个参数
   - 层数：36 层
   - 隐藏层维度：1280
   - 注意力头数：20

4. **GPT-2 XL**：
   - 参数量：15.58 亿个参数
   - 层数：48 层
   - 隐藏层维度：1600
   - 注意力头数：25

这些不同规模的模型可以根据具体的硬件资源和应用场景选择使用。较小的模型适合资源有限的情况，而较大的模型则能提供更强大的生成能力和更好的性能。

### 历史背景

- **发布日期**：GPT（Generative Pre-trained Transformer）的第一个版本，即 GPT-1，是在 2018 年由 OpenAI 发布的。具体来说，关于 GPT-1 的研究论文《Improving Language Understanding by Generative Pre-Training》在 2018 年 6 月发布。
  
- **开发动机**：GPT-2 2019年发表，是在 GPT-1 的基础上进行的重大改进。它引入了更多的参数和更大的训练数据集，显著提升了模型的能力。此外，GPT-2 还展示了强大的文本生成能力，甚至能够生成逼真的文章段落，这引发了关于 AI 伦理和社会影响的广泛讨论。

- **伦理考虑**：由于 GPT-2 的强大生成能力，OpenAI 初始时对模型的发布采取了谨慎的态度，担心其可能被滥用（例如用于生成假新闻或恶意内容）。因此，他们选择了分阶段发布，并进行了广泛的伦理讨论和研究。

- **后续发展**：随着技术的进步和社区的反馈，OpenAI 最终发布了完整的 GPT-2 模型，并开放了 API 接口。此后，许多研究人员和开发者基于 GPT-2 进行了各种创新应用，包括但不限于对话系统、自动摘要、机器翻译等。

- **后续版本**：GPT-2 的成功为后来的 GPT-3 和其他大模型的发展奠定了基础。GPT-3 进一步增加了参数量，达到了 1750 亿个参数，极大地提高了模型的表现力和多功能性。

### DNA GPT2

GPT-2 是一个重要的里程碑，在自然语言处理领域展现了强大的文本生成和理解能力。它的发布不仅推动了深度学习技术的发展,通过不断迭代和改进，GPT 系列模型已经成为现代 NLP 应用的核心工具之一。这里我们就使用GPT2网络，训练一个DNA的GPT2模型。

### 网络结构
GPT2 small的基本网络结构如下所示：


<img src="img/gpt2-stru.png" width="600px" />

In [1]:
import subprocess
import os
# 设置环境变量, autodl一般区域
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig,GPT2Tokenizer
from transformers import GPT2Tokenizer,GPT2Model,AutoModel
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import LineByLineTextDataset
from tokenizers import Tokenizer
from datasets import load_dataset

首先是分词器，我们使用的是我们上一节训练的dna_bpe_dict分词器

In [6]:
#tokenizer = GPT2Tokenizer.from_pretrained("dnagpt/dna_bpe_dict_1g")
tokenizer = GPT2Tokenizer.from_pretrained("dna_bpe_dict")
tokenizer.pad_token = tokenizer.eos_token

然后是模型的配置，我们采用的是从头训练的方法



In [7]:
max_length = 256 #最大输入长度

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=max_length, #最大长度
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = GPT2LMHeadModel(config) #for pretrain,从头预训练

## 训练数据

接着是训练数据集,最重要的是构建模型的输入和输出。

这里使用DataCollatorForLanguageModeling ，它是专为语言建模而设计（顾名思义）。除了堆叠和填充批次，它还负责创建语言模型标签——在因果语言建模中，输入也用作标签（只是移动了一个元素），并且这个数据整理器在训练期间即时创建它们，所以我们不需要复制 input_ids。

In [9]:
# 1. load dna dataset
raw_dataset = load_dataset('text',  data_files="../01-data_env/data/dna_1g.txt")
#dataset = raw_dataset["train"].select(range(100)).train_test_split(test_size=0.1, shuffle=True)
dataset = raw_dataset["train"].train_test_split(test_size=0.1, shuffle=True)

# 2. tokenize
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=max_length)

# 3. 对数据集应用分词函数
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['text'], num_proc=15)  # 设置为你的 CPU 核心数或根据需要调整

# 4. 创建一个数据收集器，用于动态填充和遮蔽
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

Map (num_proc=15):   0%|          | 0/90 [00:00<?, ? examples/s]

num_proc must be <= 10. Reducing num_proc to 10 for dataset of size 10.


Map (num_proc=10):   0%|          | 0/10 [00:00<?, ? examples/s]

In [10]:
samples = [tokenized_datasets["train"][0]]
print(samples)

[{'input_ids': [20, 1378, 237, 486, 4849, 14831, 21023, 8523, 3566, 25978, 16916, 1923, 911, 67, 637, 3261, 29515, 703, 181, 4412, 3663, 8540, 3932, 19218, 3968, 126, 3289, 14553, 198, 65, 24267, 2733, 1232, 32, 514, 919, 35, 4068, 172, 84, 7767, 1062, 527, 8291, 2514, 28, 1429, 542, 864, 137, 2492, 1922, 1744, 637, 3239, 282, 333, 1722, 120, 4419, 39, 10654, 156, 1816, 1816, 1816, 5469, 4208, 1179, 468, 112, 4596, 533, 188, 1959, 47, 1400, 64, 1986, 65, 2086, 834, 16609, 1468, 414, 34, 682, 560, 49, 3138, 14211, 2879, 16844, 122, 671, 262, 118, 1049, 347, 1003, 113, 288, 1168, 11881, 13826, 297, 90, 189, 2166, 25772, 5951, 27687, 20193, 205, 640, 50, 1082, 2015, 210, 7079, 2295, 17153, 10491, 5749, 199, 108, 25861, 372, 8448, 269, 103, 220, 243, 1150, 315, 823, 152, 9798, 229, 614, 85, 2043, 48, 234, 5146, 524, 48, 468, 12858, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [11]:
io_data = data_collator(samples)
print(io_data)

{'input_ids': tensor([[   20,  1378,   237,   486,  4849, 14831, 21023,  8523,  3566, 25978,
         16916,  1923,   911,    67,   637,  3261, 29515,   703,   181,  4412,
          3663,  8540,  3932, 19218,  3968,   126,  3289, 14553,   198,    65,
         24267,  2733,  1232,    32,   514,   919,    35,  4068,   172,    84,
          7767,  1062,   527,  8291,  2514,    28,  1429,   542,   864,   137,
          2492,  1922,  1744,   637,  3239,   282,   333,  1722,   120,  4419,
            39, 10654,   156,  1816,  1816,  1816,  5469,  4208,  1179,   468,
           112,  4596,   533,   188,  1959,    47,  1400,    64,  1986,    65,
          2086,   834, 16609,  1468,   414,    34,   682,   560,    49,  3138,
         14211,  2879, 16844,   122,   671,   262,   118,  1049,   347,  1003,
           113,   288,  1168, 11881, 13826,   297,    90,   189,  2166, 25772,
          5951, 27687, 20193,   205,   640,    50,  1082,  2015,   210,  7079,
          2295, 17153, 10491,  5749,  

这段代码展示了如何加载 DNA 数据集、对其进行分词处理，并为语言模型训练准备数据。让我们逐段解析代码，并特别关注 `DataCollatorForLanguageModeling` 函数。

### 1. 加载 DNA 数据集

```python
raw_dataset = load_dataset('text', data_files="../01-data_env/data/dna_1g.txt")
dataset = raw_dataset["train"].train_test_split(test_size=0.1, shuffle=True)
```

- **`load_dataset`**：使用 Hugging Face 的 `datasets` 库加载文本文件作为数据集。这里指定的是一个本地的 DNA 序列文本文件 `dna_1g.txt`。
- **`train_test_split`**：将原始数据集分割为训练集和测试集，其中测试集占 10%（`test_size=0.1`），并随机打乱数据（`shuffle=True`）。

### 2. 定义分词函数

```python
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=max_length)
```

- **`tokenize_function`**：这是一个自定义的分词函数，用于对数据集中的每条记录进行分词处理。
- **参数解释**：
  - `examples['text']`：获取数据集中每条记录的文本内容。
  - `truncation=True`：确保所有输入序列被截断到 `max_length` 指定的最大长度。
  - `padding='max_length'`：将所有输入序列填充到 `max_length` 指定的最大长度，以保证批次内所有序列具有相同的长度。
  - `max_length`：指定最大序列长度，需要根据具体任务和模型要求设置。

### 3. 对数据集应用分词函数

```python
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['text'], num_proc=15)
```

- **`map`**：将 `tokenize_function` 应用到整个数据集上。`batched=True` 表示批量处理，可以显著提高处理速度。
- **`remove_columns=['text']`**：分词后不再需要原始文本列，因此将其移除。
- **`num_proc=15`**：指定使用的 CPU 核心数（或进程数），可以根据你的硬件资源调整。这有助于加速分词过程。

### 4. 创建数据收集器

```python
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)
```

#### `DataCollatorForLanguageModeling` 函数详解

`DataCollatorForLanguageModeling` 是 Hugging Face 提供的一个工具，用于在训练语言模型时动态地处理批次数据。它主要用于两种任务：

- **Masked Language Modeling (MLM)**：遮蔽某些 token 并预测它们，常用于预训练模型（如 BERT）。
- **Causal Language Modeling (CLM)**：基于前文预测下一个 token，适用于生成式模型（如 GPT 系列）。

在这个例子中，`mlm=False` 表明我们正在处理因果语言建模（CLM），即每个 token 只能依赖于其前面的 token 进行预测。这对于像 GPT 这样的生成模型非常适用。

- **`tokenizer=tokenizer`**：指定用于编码和解码的分词器对象。
- **`mlm=False`**：关闭 MLM 模式，因为我们不需要遮蔽任何 token。对于因果语言建模，模型会尝试根据之前的上下文预测下一个 token。

### 开始训练

In [5]:
run_path = "gpt2_run"
train_epoches = 5
batch_size = 10


training_args = TrainingArguments(
        output_dir=run_path,
        overwrite_output_dir=True,
        num_train_epochs=train_epoches,
        per_device_train_batch_size=batch_size,
        save_steps=2000,
        save_total_limit=2,
        prediction_loss_only=True,
        fp16=True, #v100没法用
    )


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
)

[2024-12-25 01:00:37,579] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/root/minicond

In [6]:
trainer.train()
trainer.save_model("dna_gpt2_v0")

Step,Training Loss
500,8.815900
1000,8.676700
1500,8.665800
2000,8.648500
2500,8.620900
3000,8.595700
3500,8.565000
4000,8.536300
4500,8.515300
5000,8.499900


In [7]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 1509.44


In [8]:
#upload model
#model.push_to_hub("dna_gpt2_v0", organization="dnagpt", use_auth_token="hf_*******")

训练完成后，我们就可以直接使用这个模型：

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
model = AutoModel.from_pretrained('dna_gpt2_v0')
model

GPT2Model(
  (wte): Embedding(30000, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2SdpaAttention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

这里可以更清楚的看到gpt2的网络结构。 GPT-2 模型结构展示了其内部组件和层次。

### 总体架构

GPT-2 是基于 Transformer 的语言模型，采用了编码器-解码器架构中的解码器部分（即仅包含解码器）。它通过堆叠多个 Transformer 解码器层来处理输入序列，并生成输出序列。

### 详细解析

#### 1. **嵌入层 (Embedding Layers)**

```python
(wte): Embedding(30000, 768)
(wpe): Embedding(1024, 768)
```

- **`wte` (Word Token Embeddings)**：这是一个嵌入层，用于将词汇表中的 token 映射到 768 维的向量空间。`30000` 表示词汇表大小，即模型可以识别的 token 数量。
  
- **`wpe` (Positional Embeddings)**：这是位置嵌入层，用于捕捉序列中每个位置的信息。`1024` 表示模型支持的最大序列长度，`768` 表示每个位置嵌入的维度。

#### 2. **Dropout 层**

```python
(drop): Dropout(p=0.1, inplace=False)
```

- **`drop`**：这是一个 Dropout 层，用于防止过拟合。在训练过程中，它会随机丢弃一部分神经元，比例为 `p=0.1`（即 10%）。`inplace=False` 表示不会原地修改输入张量。

#### 3. **Transformer 块 (Transformer Blocks)**

```python
(h): ModuleList(
    (0-11): 12 x GPT2Block(
      ...
    )
)
```

- **`h`**：这是一个 `ModuleList`，包含了 12 个 `GPT2Block`，每个块代表一个 Transformer 解码器层。这些层是顺序堆叠的，每一层都会对输入进行进一步的变换和特征提取。

##### 每个 `GPT2Block` 的结构如下：

- **Layer Normalization (`ln_1` 和 `ln_2`)**：

  ```python
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  ```

  - **`ln_1` 和 `ln_2`**：这两个层归一化层分别位于自注意力机制之前和前馈神经网络之前。它们确保了每一层的输入具有相同的分布，从而加速训练并提高稳定性。

- **自注意力机制 (`attn`)**：

  ```python
  (attn): GPT2SdpaAttention(
    (c_attn): Conv1D(nf=2304, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  ```

  - **`c_attn`**：这是一个线性投影层，将输入映射到查询（Q）、键（K）和值（V）的空间，总维度为 `2304`（因为 Q、K、V 各占 `768` 维）。
  - **`c_proj`**：这是另一个线性投影层，将自注意力机制的结果重新映射回原始维度 `768`。
  - **`attn_dropout` 和 `resid_dropout`**：这两个 Dropout 层分别应用于注意力得分和残差连接，以防止过拟合。

- **前馈神经网络 (`mlp`)**：

  ```python
  (mlp): GPT2MLP(
    (c_fc): Conv1D(nf=3072, nx=768)
    (c_proj): Conv1D(nf=768, nx=3072)
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  ```

  - **`c_fc`**：这是一个线性投影层，将输入映射到更大的维度 `3072`，以增加表达能力。
  - **`c_proj`**：这是另一个线性投影层，将结果重新映射回原始维度 `768`。
  - **`act`**：激活函数，使用的是改进版的 GELU（Gaussian Error Linear Unit），有助于非线性变换。
  - **`dropout`**：用于防止过拟合的 Dropout 层。

#### 4. **最终的 Layer Normalization**

```python
(ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
```

- **`ln_f`**：这是最后一个 Layer Normalization 层，应用于所有 Transformer 块之后的输出。它确保了最终输出的稳定性和一致性。

### 工作流程

1. **输入处理**：
   - 输入 token 通过 `wte` 转换为词嵌入。
   - 输入位置通过 `wpe` 转换为位置嵌入。
   - 词嵌入和位置嵌入相加，然后通过 `drop` 进行 Dropout。

2. **多层 Transformer 解码器**：
   - 输入经过 12 层 `GPT2Block`，每层包括自注意力机制和前馈神经网络。
   - 每层之间有残差连接（Residual Connection），并且在每个子模块前后都有 Layer Normalization。

3. **最终输出**：
   - 最终输出通过 `ln_f` 进行归一化，得到模型的隐状态表示。


当然，我们也可以使用Netron等工具，查看网络结构

<img src="img/gpt2-netron.png" width="600px" />


当然，还有一些更酷炫的：
https://bbycroft.net/llm

<img src="img/llm-visual.png" width="600px" />

In [5]:
tokenizer = AutoTokenizer.from_pretrained('dna_bpe_dict')
tokenizer.tokenize("GAGCACATTCGCCTGCGTGCGCACTCACACACACGTTCAAAAAGAGTCCATTCGATTCTGGCAGTAG")

dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
inputs = tokenizer(dna, return_tensors = 'pt')
print(inputs)

outputs = model(inputs["input_ids"])
print(list(outputs))

{'input_ids': tensor([[    1,   191,    29,   753,  1241,  2104, 12297,   357,    85,  4395,
         26392,    16]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
['last_hidden_state', 'past_key_values']


In [4]:
#为了使用方便，我们一般还会把分词器也存到模型目录下
tokenizer.save_pretrained("dna_gpt2_v0")

('dna_gpt2_v0/tokenizer_config.json',
 'dna_gpt2_v0/special_tokens_map.json',
 'dna_gpt2_v0/vocab.json',
 'dna_gpt2_v0/merges.txt',
 'dna_gpt2_v0/added_tokens.json',
 'dna_gpt2_v0/tokenizer.json')

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("dna_bpe_dict")
tokenizer.pad_token = tokenizer.eos_token

In [5]:
from transformers import AutoTokenizer, AutoModel
model = AutoModel.from_pretrained('dna_gpt2_v0')

In [6]:
model.push_to_hub("dnagpt/dna_gpt2_v0",  token="hf_***")

[2024-12-30 20:29:16,315] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/root/minicond

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dnagpt/dna_gpt2_v0/commit/e7c5a5c59e28329114d3ab64cb7b585f198b1f5f', commit_message='Upload model', commit_description='', oid='e7c5a5c59e28329114d3ab64cb7b585f198b1f5f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dnagpt/dna_gpt2_v0', endpoint='https://huggingface.co', repo_type='model', repo_id='dnagpt/dna_gpt2_v0'), pr_revision=None, pr_num=None)

In [7]:
tokenizer.push_to_hub("dnagpt/dna_gpt2_v0",  token="hf_**")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dnagpt/dna_gpt2_v0/commit/16138639cb17307b84421e443a1c67f4fe188121', commit_message='Upload tokenizer', commit_description='', oid='16138639cb17307b84421e443a1c67f4fe188121', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dnagpt/dna_gpt2_v0', endpoint='https://huggingface.co', repo_type='model', repo_id='dnagpt/dna_gpt2_v0'), pr_revision=None, pr_num=None)